# Pràctica 3 - PLH

### Realitzada pels alumnes Lluc Furriols i Pau Prat Moreno

In [1]:
'''
import os 
f = open("/dev/null", "w")
os.dup2(f.fileno(), 2)
f.close()

import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()
'''

'\nimport os \nf = open("/dev/null", "w")\nos.dup2(f.fileno(), 2)\nf.close()\n\nimport nltk\nimport ssl\n\ntry:\n    _create_unverified_https_context = ssl._create_unverified_context\nexcept AttributeError:\n    pass\nelse:\n    ssl._create_default_https_context = _create_unverified_https_context\n\nnltk.download()\n'

### Importació de llibreries

In [2]:
import nltk
nltk.download('punkt', quiet=True) # Tokenitzador
nltk.download('averaged_perceptron_tagger', quiet=True) # Etiquetador POS
nltk.download('maxent_ne_chunker', quiet=True) # Etiquetador Entitats Anomenades
nltk.download('words', quiet=True)


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1002)>
[nltk_data] Error loading maxent_ne_chunker: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

### Carreguem les dades

In [3]:
nltk.download('conll2002')
from nltk.corpus import conll2002

train_esp = conll2002.iob_sents('esp.train') # Train, 
val_esp = conll2002.iob_sents('esp.testa') # Val
test_esp = conll2002.iob_sents('esp.testb') # Test

train_ned = conll2002.iob_sents('ned.train') # Train
val_ned = conll2002.iob_sents('ned.testa') # Val
test_ned = conll2002.iob_sents('ned.testb') # Test

[nltk_data] Error loading conll2002: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


### Funcions essencials

In [4]:
def get_token(sequence):
    """
    Retorna una llista de tokens.
    """
    return [[(token) for token, pos, entity in sentence] for sentence in sequence]

def get_token_POS(sequence):
    """
    Retorna una llista de tokens i el seu POS tag.
    """
    return [[(token, pos) for token, pos, entity in sentence] for sentence in sequence]

def get_token_entity(sequence):
    """
    Retorna una llista de tokens i les seves entitats.
    """
    return [[(token, entity) for token, pos, entity in sentence] for sentence in sequence]

In [5]:
def extract_entities(tagged_words, encoding='BIO'):
    """
    Extreu les entitats d'una llista de paraules etiquetades segons l'encoding especificat.
    
    Arguments:
        tagged_words: Una llista de tuples (word, tag), on 'word' és una paraula del text i 'tag' és la seva etiqueta (BIO/BIOW/IO).
        encoding: El tipus de codificació utilitzat per les etiquetes ('BIO', 'BIOW', 'IO').
        
    Retorna:
        Una llista de tuples (index_inicial, index_final, etiqueta) que representen les entitats trobades.
    """
    entities = []  # Llista on es guardaran les entitats trobades
    current_entity = []  # Guarda les paraules de l'entitat actual
    current_type = None  # Tipus de l'entitat actual
    current_start_index = None  # Índex d'inici de l'entitat actual

    # Recorre cada paraula i la seva etiqueta en la llista de paraules etiquetades
    for index, (word, tag) in enumerate(tagged_words):
        # Determina el tipus de tag, o 'None' si el tag és 'O'
        tag_type = None if tag == 'O' else tag[2:]
        # Obté el prefix del tag, o 'None' si és 'O'
        tag_prefix = tag[0] if tag != 'O' else None

        # Finalitza l'entitat actual si es troba un tag 'O' o si canvia el tipus d'entitat
        if tag == 'O' or tag_prefix not in ['B', 'I', 'W'] or (tag_prefix == 'I' and tag_type != current_type):
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
                current_entity = []
                current_type = None
        if tag == 'O':
            continue

        # Gestiona els canvis d'entitat segons l'encoding
        if encoding == 'IO' and tag_type != current_type:
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]
            current_start_index = index
            current_type = tag_type
        elif tag_prefix in ['B', 'W']:
            if current_entity:
                entities.append((current_start_index, index - 1, current_type))
            current_entity = [word]
            current_start_index = index
            current_type = tag_type
            if tag_prefix == 'W':
                # Si el tag és 'W-', afegeix immediatament com a entitat de paraula única
                entities.append((current_start_index, index, current_type))
                current_entity = []
                current_type = None
        elif tag_prefix == 'I' and current_type == tag_type:
            current_entity.append(word)

    # Afegeix la darrera entitat si encara hi ha una pendent
    if current_entity:
        entities.append((current_start_index, index, current_type))

    return entities

In [6]:
def evaluate_entities(true_entities, pred_entities):
    """
    Avaluació de les entitats reconegudes comparant conjunts d'entitats.

    Args:
    true_entities (list): Llista de tuples representant les entitats reals (start, end, type).
    pred_entities (list): Llista de tuples representant les entitats predites (start, end, type).

    Returns:
    dict: Un diccionari amb les mètriques 'precision', 'recall', i 'f1_score'.
    """
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for true_sent, pred_sent in zip(true_entities, pred_entities):
        true_set = set(true_sent)
        pred_set = set(pred_sent)

        true_positives += len(true_set & pred_set)
        false_positives += len(pred_set - true_set)
        false_negatives += len(true_set - pred_set)

    precisio = true_positives / (true_positives + false_positives) if true_positives + false_positives > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives > 0 else 0
    f1_score = 2 * precisio * recall / (precisio + recall) if (precisio + recall) > 0 else 0

    # Arrodonim els valors a 3 decimals
    precisio = round(precisio, 3)
    recall = round(recall, 3)
    f1_score = round(f1_score, 3)

    return {
        'precisio': precisio,
        'recall': recall,
        'f1_score': f1_score
    }
    

### Creem un model per predir els postags:

In [7]:
from crf import *
import re
import string

class FeatureGetterPOS:
    def __init__(self):
        # Compila un patró regex per detectar dígits
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Obté el prefix del token.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Obté el sufix del token.
        """
        return token[-n:] if len(token) > n else token
    

    def get_features(self, tokens, index):
        """
        Obté les característiques del token.
        """
        token = tokens[index]
        features = ["WORD_" + token]

        if token[0].isupper():
            features.append("CAPITALIZATION")

        if self.has_digit(token):
            features.append("HAS_NUM")

        if self.has_punctuation(token):
            features.append("PUNCTUATION")
            
        features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])
        features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])
        if index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)
        if index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

IMPORTANT: La següent cel·la de codi és computacionalment costosa d'executar, és per això que a  continuació carreguem el model ja entrenat prèviament, per tal d'estalviar-nos l'execució d'aquesta cel·la.

In [8]:
'''# Obtenim les dades d'entrenament amb les etiquetes POS
train_esp_pos = get_token_POS(train_esp)

# Creem i configurem el model
feature_getter_pos = FeatureGetterPOS()
ct_POS = CRFTagger(feature_func=feature_getter_pos.get_features)

# Entrenem el model
ct_POS.train(train_esp_pos, 'model.crf.taggerPOS')'''

"# Obtenim les dades d'entrenament amb les etiquetes POS\ntrain_esp_pos = get_token_POS(train_esp)\n\n# Creem i configurem el model\nfeature_getter_pos = FeatureGetterPOS()\nct_POS = CRFTagger(feature_func=feature_getter_pos.get_features)\n\n# Entrenem el model\nct_POS.train(train_esp_pos, 'model.crf.taggerPOS')"

In [9]:
from nltk.tag import CRFTagger

feature_getter_pos = FeatureGetterPOS()

# Creem una instància del CRFTagger
ct_POS = CRFTagger(feature_func=feature_getter_pos.get_features)

# Carreguem el model previament entrenat
ct_POS.set_model_file('model.crf.taggerPOS')

In [10]:
# Funció per avaluar el POS tagging
def evaluate_POS(true_tags, pred_tags):
    """
    Avaluació del POS tagging comparant seqüències de tags.

    Args:
    true_tags (list): Llista de llistes amb els tags reals.
    pred_tags (list): Llista de llistes amb els tags predits.

    Returns:
    dict: Un diccionari amb les mètriques 'accuracy'.
    """
    total = 0
    correct = 0

    for true_sent, pred_sent in zip(true_tags, pred_tags):
        total += len(true_sent)
        correct += sum(1 for true_tag, pred_tag in zip(true_sent, pred_sent) if true_tag == pred_tag)

    accuracy = correct / total if total > 0 else 0

    return accuracy
  

# Prediccions del POS tagging
y_pred_POS = ct_POS.tag_sents(get_token(test_esp))
y_real_POS = get_token_POS(test_esp)
print(evaluate_POS(y_real_POS, y_pred_POS))


0.9614033725962005


El model predeix el postags amb una accuracy de 96%. Com que és un bon resultat, l'utilitzarem en les feature functions

## Model per a predir entitats

Aquesta classe és igual que la anterior, però utilitza el model anterior per predir els postags

In [11]:
import re
import string
class FeatureGetter:
    def __init__(self, ct_POS, use_digit=True, use_punctuation=True, use_prefix=True, use_suffix=True, use_capitalization=True, use_prev_word=True, use_next_word=True):
        """
        Inicialitza el FeatureGetter amb opcions per activar o desactivar certes característiques.
        """
        self.ct_POS = ct_POS
        self.use_digit = use_digit
        self.use_punctuation = use_punctuation
        self.use_prefix = use_prefix
        self.use_suffix = use_suffix
        self.use_capitalization = use_capitalization
        self.use_prev_word = use_prev_word
        self.use_next_word = use_next_word
        self._pattern = re.compile(r"\d")

    def has_digit(self, token):
        """
        Comprova si el token conté algun dígit.
        """
        return any(char.isdigit() for char in token)

    def has_punctuation(self, token):
        """
        Comprova si el token conté algun signe de puntuació.
        """
        return any(char in string.punctuation for char in token)

    def get_prefix(self, token, n=3):
        """
        Obté el prefix del token.
        """
        return token[:n] if len(token) > n else token

    def get_suffix(self, token, n=3):
        """
        Obté el sufix del token.
        """
        return token[-n:] if len(token) > n else token
    
    def pos_tag(self, token):
        """
        Etiqueta POS del token.
        """
        return self.ct_POS.tag([token])[0][1]


    def get_features(self, tokens, index):
        """
        Obté les característiques del token.
        """
        token = tokens[index]
        features = ["WORD_" + token]

        if self.use_capitalization and token[0].isupper():
            features.append("CAPITALIZATION")

        if self.use_digit and self.has_digit(token):
            features.append("HAS_NUM")

        if self.use_punctuation and self.has_punctuation(token):
            features.append("PUNCTUATION")

        if self.use_suffix:
            features.extend(["SUF_" + self.get_suffix(token, n) for n in range(1, 4) if len(token) >= n])

        if self.use_prefix:
            features.extend(["PRE_" + self.get_prefix(token, n) for n in range(1, 4) if len(token) >= n])

        pos_tag = self.pos_tag(token)
        features.append("POS_" + pos_tag)
        if self.use_prev_word and index > 0:
            prev_token = tokens[index - 1]
            features.append("PREV_WORD_" + prev_token)

        if self.use_next_word and index < len(tokens) - 1:
            next_token = tokens[index + 1]
            features.append("NEXT_WORD_" + next_token)
        return features

### Grid Search

In [12]:
from itertools import product
from nltk.tag import CRFTagger

def grid_search_feature_combinations(train_data, val_data, feature_options):
    """
    Realitza un grid search per trobar la millor combinació de funcions de característiques que maximitza el recall.
    
    Args:
    train_data (list): Dades d'entrenament en format BIO.
    val_data (list): Dades de validació en format BIO.
    feature_options (dict): Un diccionari amb noms de les característiques com a claus i True/False com a valors per defecte.
    
    Returns:
    tuple: La millor combinació de característiques i el seu valor de recall.
    """
    best_recall = 0
    best_combination = None

    # Genera totes les combinacions possibles de True/False per a cada característica
    keys = feature_options.keys()
    train_data = get_token_entity(train_data) 
    
    for values in product([True, False], repeat=len(keys)):
        options = dict(zip(keys, values))
        feature_getter = FeatureGetter(**options, ct_POS=ct_POS)
        ct = CRFTagger(feature_func=feature_getter.get_features)
        
        # Entrena el model CRFTagger amb les dades d'entrenament
        ct.train(train_data, 'model.crf.tagger')

        # Prepara les dades de test i obtenir les prediccions
        y_real = [extract_entities(sent) for sent in get_token_entity(val_data)]
        val_tokens = get_token(val_data)
        y_pred = ct.tag_sents(val_tokens)
        y_pred_entities = [extract_entities(sent) for sent in y_pred]

        # Avaluació de les entitats predites respecte a les reals
        metrics = evaluate_entities(y_real, y_pred_entities)
        recall = metrics['recall']
        # Actualitza la millor combinació si el recall actual és millor
        if recall > best_recall:
            best_recall = recall
            best_combination = options

    return best_combination, best_recall

# Defineix les opcions de les característiques
feature_options = {
    'use_prefix': True,
    'use_suffix': True,
    'use_prev_word': True,
    'use_next_word': True
}

#best_combination, best_recall = grid_search_feature_combinations(train_esp, val_esp, feature_options)
#print("Best feature combination:", best_combination)
#print("Best recall:", best_recall)

La cel·la anterior té un temps d'execució molt elevat, per la qual cosa si es desitja executar, només cal descomentar les últimes tres línies.
Per tal d'estalviar tot aquest temps de còmput, els resultats obtinguts són els següents:

* Best feature combination: {'use_prefix': True, 'use_suffix': True, 'use_prev_word': True, 'use_next_word': True}
* Best recall: 0.711

Com podem veure, la combinació que dona un recall més elevat és amb totes les feature functions actives, per la qual cosa les utilitzarem en el model final.

## Codificació BIO

In [13]:
from nltk.tag import CRFTagger
import pycrfsuite

ct_basic = CRFTagger(feature_func=None)

# Train and test sets without the postag
train_esp_first = get_token_entity(train_esp)
test_esp_first = get_token_entity(test_esp)
print("Quina forma tenen les nostres dades d'entrenament: ",train_esp_first[0])

ct_basic.train(train_esp_first, 'model.crf.tagger')

Quina forma tenen les nostres dades d'entrenament:  [('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


In [14]:
# Obtenim les dades d'entrenament, que estan en format BIO
train_data = get_token_entity(train_esp)  

# Creació del model
feature_getter = FeatureGetter(ct_POS=ct_POS)
ct = CRFTagger(feature_func=feature_getter.get_features)

# Entrenament del model
ct.train(train_data, 'model.crf.tagger')

# Preparació de les dades de prova i predicció
y_real = get_token_entity(test_esp)
y_pred = ct.tag_sents(get_token(test_esp))

# Extraiem les entitats
y_real_entities = [extract_entities(sent) for sent in y_real]
y_pred_entities = [extract_entities(sent) for sent in y_pred]

# Avaluem les entitats
metrics = evaluate_entities(y_real_entities, y_pred_entities)
print(f'Precisio: {metrics["precisio"]:.3f}')
print(f'Recall: {metrics["recall"]:.3f}')
print(f'F1 Score: {metrics["f1_score"]:.3f}')

Precisio: 0.784
Recall: 0.769
F1 Score: 0.776


## Codificació IO

In [15]:
def bio_to_io(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a IO.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, io_tag) on 'io_tag' és en codificació IO per cada frase.
    """
    io_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        io_tagged_sentence = []
        for word, tag in sentence:
            if tag.startswith('B-'):
                # Canvia B- per I-
                io_tagged_sentence.append((word, 'I-' + tag[2:]))
            elif tag.startswith('I-'):
                io_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                io_tagged_sentence.append((word, 'O'))
        io_tagged_sentences.append(io_tagged_sentence)
    
    return io_tagged_sentences


tagged_words = get_token_entity(train_esp)
tagged_words_io = bio_to_io(tagged_words)

print(f"BIO:{tagged_words[3]}")
print(f"IO:{tagged_words_io[3]}")

entities = extract_entities(tagged_words_io[3], "IO")
print(f"Entities IO: {entities}")

BIO:[('La', 'O'), ('petición', 'O'), ('del', 'O'), ('Abogado', 'B-PER'), ('General', 'I-PER'), ('tiene', 'O'), ('lugar', 'O'), ('después', 'O'), ('de', 'O'), ('que', 'O'), ('un', 'O'), ('juez', 'O'), ('del', 'O'), ('Tribunal', 'B-ORG'), ('Supremo', 'I-ORG'), ('del', 'O'), ('estado', 'O'), ('de', 'O'), ('Victoria', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), ('se', 'O'), ('viera', 'O'), ('forzado', 'O'), ('a', 'O'), ('disolver', 'O'), ('un', 'O'), ('jurado', 'O'), ('popular', 'O'), ('y', 'O'), ('suspender', 'O'), ('el', 'O'), ('proceso', 'O'), ('ante', 'O'), ('el', 'O'), ('argumento', 'O'), ('de', 'O'), ('la', 'O'), ('defensa', 'O'), ('de', 'O'), ('que', 'O'), ('las', 'O'), ('personas', 'O'), ('que', 'O'), ('lo', 'O'), ('componían', 'O'), ('podían', 'O'), ('haber', 'O'), ('obtenido', 'O'), ('información', 'O'), ('sobre', 'O'), ('el', 'O'), ('acusado', 'O'), ('a', 'O'), ('través', 'O'), ('de', 'O'), ('la', 'O'), ('página', 'O'), ('CrimeNet', 'B-MISC'), ('.', 'O')]
IO:[('La'

Entrenar el model amb la codificació IO i avaluar els resultats

In [16]:
# Convertim les dades d'entrenament de format BIO a IO
train_data = bio_to_io(get_token_entity(train_esp))

# Creem el model i l'entrenem amb les dades d'entrenament
feature_getter = FeatureGetter(ct_POS=ct_POS)
ct = CRFTagger(feature_func=feature_getter.get_features)
ct.train(train_data, 'model.crf.tagger')

# Convertim les dades de prova de format BIO a IO i fem les prediccions 
y_real = bio_to_io(get_token_entity(test_esp))
y_pred = bio_to_io(ct.tag_sents(get_token(test_esp)))

# Extraiem les entitats reals i predites
y_real_entities = [extract_entities(sent, "IO") for sent in y_real]
y_pred_entities = [extract_entities(sent, "IO") for sent in y_pred]

# Avaluem les entitats 
evaluate_entities(y_real_entities, y_pred_entities)

{'precisio': 0.785, 'recall': 0.763, 'f1_score': 0.774}

## Codificació BIOW

In [17]:
def bio_to_biow(bio_tagged_sentences):
    """
    Converteix les etiquetes de múltiples frases des de la codificació BIO a BIOW.
    
    Arguments:
        bio_tagged_sentences: Una llista de llistes de tuples (word, tag) on 'tag' és en codificació BIO.
    
    Retorna:
        Una llista de llistes de tuples (word, biow_tag) on 'biow_tag' és en codificació BIOW per cada frase.
    """
    biow_tagged_sentences = []
    
    for sentence in bio_tagged_sentences:
        biow_tagged_sentence = []
        for i, (word, tag) in enumerate(sentence):
            if tag.startswith('B-'):
                # Comprovar si l'entitat només té una paraula (W)
                if i + 1 < len(sentence) and sentence[i + 1][1].startswith('I-'):
                    biow_tagged_sentence.append((word, 'B-' + tag[2:]))
                else:
                    biow_tagged_sentence.append((word, 'W-' + tag[2:]))
            elif tag.startswith('I-'):
                biow_tagged_sentence.append((word, tag))
            else:
                # Manté les etiquetes 'O' tal com estan
                biow_tagged_sentence.append((word, 'O'))
        biow_tagged_sentences.append(biow_tagged_sentence)
    
    return biow_tagged_sentences


tagged_words = get_token_entity(train_esp)
tagged_words_io = bio_to_biow(tagged_words)
print(f"BIO:{tagged_words[0]}")
print(f"BIOW:{tagged_words_io[0]}")

entities = extract_entities(tagged_words_io[3], "BIOW")
print(f"Entities BIOW: {entities}")

BIO:[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]
BIOW:[('Melbourne', 'W-LOC'), ('(', 'O'), ('Australia', 'W-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'W-ORG'), (')', 'O'), ('.', 'O')]
Entities BIOW: [(3, 4, 'PER'), (13, 14, 'ORG'), (18, 18, 'LOC'), (20, 20, 'LOC'), (59, 59, 'MISC')]


Entrenar el model amb la codificació BIOW i avaluar els resultats

In [18]:
# Convertim les dades d'entrenament de format BIO a BIOW
train_data = bio_to_biow(get_token_entity(train_esp))  

# Creem una instància del CRFTagger i l'entrenem 
feature_getter = FeatureGetter(ct_POS=ct_POS)
ct = CRFTagger(feature_func=feature_getter.get_features)
ct.train(train_data, 'model.crf.tagger')

# Convertim les dades de prova de format BIO a BIOW i fem les prediccions 
y_real = bio_to_biow(get_token_entity(test_esp))
y_pred = bio_to_biow(ct.tag_sents(get_token(test_esp)))

# Extraiem les entitats reals i predites
y_real_entities = [extract_entities(sent, "BIOW") for sent in y_real]
y_pred_entities = [extract_entities(sent, "BIOW") for sent in y_pred]

# Avaluem les entitats
evaluate_entities(y_real_entities, y_pred_entities)

{'precisio': 0.792, 'recall': 0.282, 'f1_score': 0.416}